<a href="https://colab.research.google.com/github/Rachit180/FER/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
 from google.colab import drive
 drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -q "/content/drive/MyDrive/WESAD (1).zip"


replace WESAD/S10/S10.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import keras
from matplotlib import pyplot
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adamax,Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D, Activation, Input, ZeroPadding2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

from tensorflow.keras.models import load_model

In [ ]:
import os
files = os.listdir(
    '/content/WESAD/'
)
for file in files:
  if file.endswith('.pdf') : continue
  for d in os.listdir(f'/content/WESAD/{file}'):
    if d.endswith('.csv'):
      data = pd.read_csv(f'/content/WESAD/{file}/{d}')
      break


In [ ]:
data

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Base directory containing subject folders
base_path = '/content/WESAD/'

# Initialize lists for features and labels
X_data = []
y_data = []

# Iterate through all subject folders
subject_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]

for subject in subject_folders:
    try:
        subject_path = os.path.join(base_path, subject)

        # Load synchronized data and labels from .pkl file with specified encoding
        with open(os.path.join(subject_path, f"{subject}.pkl"), 'rb') as f:
            data = pickle.load(f, encoding='latin1')  # Change encoding if necessary

        # Extract chest sensor data
        chest_acc = data['signal']['chest']['ACC']  # Accelerometer (x, y, z)
        chest_eda = data['signal']['chest']['EDA']  # Electrodermal activity
        chest_temp = data['signal']['chest']['TEMP']  # Temperature

        # Extract wrist sensor data
        wrist_acc = data['signal']['wrist']['ACC']  # Accelerometer (x, y, z)
        wrist_eda = data['signal']['wrist']['EDA']  # Electrodermal activity
        wrist_temp = data['signal']['wrist']['TEMP']  # Temperature

        # Combine all features from chest and wrist sensors
        combined_features = np.hstack((
            chest_acc, chest_eda[:, np.newaxis], chest_temp[:, np.newaxis],
            wrist_acc, wrist_eda[:, np.newaxis], wrist_temp[:, np.newaxis]
        ))

        # Standardize features
        scaler = StandardScaler()
        standardized_features = scaler.fit_transform(combined_features)

        # Extract labels
        labels = data['label']  # Stress levels: 0=undefined, 1=baseline, 2=stress, etc.

        # Append to main dataset
        X_data.append(standardized_features)
        y_data.append(labels)

    except Exception as e:
        print(f"Error processing subject {subject}: {e}")

# Ensure there's valid data to process
if X_data and y_data:
    # Concatenate data from all subjects
    X_data = np.concatenate(X_data, axis=0)
    y_data = np.concatenate(y_data, axis=0)

    # Pad sequences for RNN models
    X_data_padded = pad_sequences(X_data, padding='post', dtype='float32')

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_data_padded, y_data, test_size=0.2, random_state=42, stratify=y_data)

    # Display data shapes
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_test shape:", y_test.shape)
else:
    print("No valid data was processed.")



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = '/content/WESAD/S15/S15_quest.csv'

# Read the file with appropriate delimiter
data = pd.read_csv(file_path, header=None, delimiter=';')

# Remove metadata rows (e.g., rows starting with # or empty rows)
data = data[~data[0].astype(str).str.startswith('#')]
data = data.dropna(how='all')  # Drop completely empty rows

# Reset index after filtering
data.reset_index(drop=True, inplace=True)

# Debugging: Check the dataset structure
print("Cleaned Data Sample:")
print(data.head())

# Extract sections relevant for features (X) and labels (Y)
sections = {
    'PANAS': data[data[0].str.contains('PANAS', na=False)].iloc[:, 1:].astype(float).values,
    'STAI': data[data[0].str.contains('STAI', na=False)].iloc[:, 1:].astype(float).values,
    'DIM': data[data[0].str.contains('DIM', na=False)].iloc[:, 1:].astype(float).values,
    'SSSQ': data[data[0].str.contains('SSSQ', na=False)].iloc[:, 1:].astype(float).values,
}

# Combine features (X) from all sections
X_data = np.hstack([sections['PANAS'], sections['STAI'], sections['DIM'], sections['SSSQ']])

# Create labels (Y)
# Example: Use `SSSQ` as stress levels or assign custom labels based on rows
# For simplicity, we label high values in SSSQ (e.g., mean > 3) as stress
SSSQ_mean = np.mean(sections['SSSQ'], axis=1)
y_data = np.where(SSSQ_mean > 3, 1, 0)  # Binary labels: 1 = Stress, 0 = No Stress

# Debugging: Check shapes
print(f"Feature shape: {X_data.shape}, Label shape: {y_data.shape}")

# Standardize features
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42, stratify=y_data)

# Debugging: Display data splits
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Ensure the data is in the correct shape for RNN
# RNN expects data in the format (samples, timesteps, features)
# Assume each row is treated as one timestep for simplicity
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Define the RNN model
def create_rnn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(64, input_shape=input_shape, activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(32, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification, use sigmoid activation

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize the model
input_shape = (X_train_rnn.shape[1], X_train_rnn.shape[2])
model = create_rnn_model(input_shape)

# Display model summary
model.summary()

# Train the model
history = model.fit(X_train_rnn, y_train,
                    validation_split=0.2,
                    epochs=20,
                    batch_size=32,
                    verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_rnn, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on test data
y_pred = model.predict(X_test_rnn)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()

# Generate classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Ensure the data is in the correct shape for RNN
# RNN expects data in the format (samples, timesteps, features)
# Assume each row is treated as one timestep for simplicity
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Define the RNN model
def create_rnn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(64, input_shape=input_shape, activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(32, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification, use sigmoid activation

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize the model
input_shape = (X_train_rnn.shape[1], X_train_rnn.shape[2])
model = create_rnn_model(input_shape)

# Display model summary
model.summary()

# Train the model
history = model.fit(X_train_rnn, y_train,
                    validation_split=0.2,
                    epochs=20,
                    batch_size=32,
                    verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_rnn, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on test data
y_pred = model.predict(X_test_rnn)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()

# Generate classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))


In [ ]:
import os
import pandas as pd

# Initialize an empty DataFrame for storing the processed data
processed_data = pd.DataFrame()

# Loop through the files in the directory
files = os.listdir('/content/WESAD/')

for file in files:
    if file.endswith('.pdf'):  # Skip any .pdf files
        continue

    for d in os.listdir(f'/content/WESAD/{file}'):
        if d.endswith('.csv'):  # Process only .csv files
            # Read the CSV file with explicit delimiter ';'
            try:
                data = pd.read_csv(f'/content/WESAD/{file}/{d}', header=None, delimiter=';')

                # Debug: Check the first few rows to ensure data is loaded correctly
                print(f"Loaded {d} from {file}")
                print(data.head())  # Display the first few rows of the file

                # Step 1: Remove rows that are just separators (e.g., rows with only ';')
                data = data[~data.apply(lambda row: row.astype(str).str.contains(';;;;', regex=False).all(), axis=1)]

                # Step 2: Remove rows that start with '#', as they are metadata or comments
                data = data[~data[0].str.startswith('#', na=False)]

                # Step 3: Reset the columns if any rows are removed
                if not data.empty:
                    data.reset_index(drop=True, inplace=True)

                    # Step 4: Assign column names (if there are more columns)
                    data.columns = ['col' + str(i) for i in range(data.shape[1])]

                    # Step 5: Convert columns to appropriate types (e.g., numeric)
                    data.iloc[:, 1:] = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

                    # Step 6: Handle missing values (example: fill NaN with the mean of the column)
                    data.fillna(data.mean(), inplace=True)

                    # Append the processed data to the final DataFrame
                    processed_data = pd.concat([processed_data, data], ignore_index=True)

            except Exception as e:
                print(f"Error reading file {d}: {e}")

# Display the cleaned data
print(processed_data.head())


In [ ]:
import os
import pandas as pd

# Path to the root directory containing the subdirectories
base_path = "/content/WESAD/"

# Get a list of all files and directories in the base path
files = os.listdir(base_path)

# Initialize a dictionary to store the data
data = {}

# Iterate through each file/subdirectory
for file in files:
    # Skip files that are PDFs
    if file.endswith('.pdf'):
        continue

    # Iterate through the contents of the subdirectories
    for d in os.listdir(os.path.join(base_path, file)):
        # If the file is a CSV
        if d.endswith('.csv'):
            file_path = os.path.join(base_path, file, d)
            print(f"Loading {d} from {file_path}")

            # Read the CSV file, skipping metadata rows (first 2 rows in this case)
            temp_data = pd.read_csv(file_path, skiprows=2)

            # Drop any unnecessary columns (like those that only contain NaN values)
            temp_data.dropna(axis=1, how='all', inplace=True)

            # Select only numeric columns for mean imputation
            numeric_data = temp_data.select_dtypes(include=['number'])

            # Fill NaN values with column means in numeric columns
            temp_data[numeric_data.columns] = numeric_data.fillna(numeric_data.mean())

            # Store the cleaned data in the dictionary
            data[d] = temp_data

            # Output first 5 rows to confirm the data
            print(temp_data.head())

# After cleaning, you can access the cleaned data like this:
# data["S17_quest.csv"], data["S16_quest.csv"], etc.


In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Example preprocessing function for one file
def preprocess_data(data):
    # Ensure the `# ORDER` column exists
    if '# ORDER' not in data.columns:
        raise KeyError("The '# ORDER' column is missing in the dataset.")

    # Select relevant numeric columns as features (all except the first column)
    features = data.iloc[:, 1:].select_dtypes(include=['number']).values  # Numeric columns only
    labels = data['# ORDER'].values  # Use the `# ORDER` column as labels

    # Standardize the features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    # Pad sequences (if your data contains varying sequence lengths)
    features = pad_sequences(features, padding='post', dtype='float32')

    return features, labels

# Prepare the data (assuming 'data' dictionary is already defined)
X_data = []
y_data = []

for file_name, temp_data in data.items():
    try:
        X, y = preprocess_data(temp_data)
        X_data.append(X)
        y_data.append(y)
    except KeyError as e:
        print(f"Skipping file '{file_name}': {e}")

# Convert to numpy arrays (if any valid data was processed)
if X_data and y_data:
    X_data = np.concatenate(X_data, axis=0)
    y_data = np.concatenate(y_data, axis=0)

    # Check the shape of the prepared data
    print("Prepared data shapes:", X_data.shape, y_data.shape)
else:
    print("No valid data was processed.")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Encode labels into integers and one-hot encode
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_data)
y_one_hot = to_categorical(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_one_hot, test_size=0.2, random_state=42)

# Build the RNN model
model = Sequential([
    tf.keras.layers.Masking(mask_value=0.0, input_shape=(X_train.shape[1], X_train.shape[2])),
    SimpleRNN(128, activation='relu', return_sequences=True),
    Dropout(0.3),
    SimpleRNN(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Output layer with softmax for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Save the model for future use
model.save('rnn_model.h5')

# Plot training history
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()


In [ ]:
# Assuming the dataset has columns 'ACC', 'EDA', 'TEMP', and the target column is 'label'
# Example: You have a dataframe `data` with these columns
# Split the features and target

X_data = data[['ACC', 'EDA', 'TEMP']]  # Select feature columns
y_data = data['label']  # Select the target column (e.g., 'label')


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Ensure the data is in the correct shape for RNN
# RNN expects data in the format (samples, timesteps, features)
# Assume each row is treated as one timestep for simplicity
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Define the RNN model
def create_rnn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(64, input_shape=input_shape, activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(32, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification, use sigmoid activation

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize the model
input_shape = (X_train_rnn.shape[1], X_train_rnn.shape[2])
model = create_rnn_model(input_shape)

# Display model summary
model.summary()

# Train the model
history = model.fit(X_train_rnn, y_train,
                    validation_split=0.2,
                    epochs=20,
                    batch_size=32,
                    verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_rnn, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on test data
y_pred = model.predict(X_test_rnn)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()

# Generate classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))


In [ ]:
print("Accuracy of our model on test data : " , model.evaluate(X_test,y_test)[1]*100 , "%")